In [18]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
import pandas as pd
import numpy as np

import time

In [19]:
def click_btn(clickable):
    for btn in clickable:
        try:
            btn.click()
        except:
            pass

## Project data
The entire dataset I'm using in this project is from https://growdiaries.com/, as one of the significant weed-growing communities in the world, it holds a large amount of data. Each row represents a data point, each column represents one of the diary features.

### Step 1:
How to get the details from all these diaries? 
I started with a naive approach, trying to load 10K of diaries using the website UI wasn't a great idea. So I took a deep look into the query of the API requests the website is making. Using Postman I realized that the query has a limit of 2K diaries and it has a starting point and count of how many rows should the server retrieve to the user. Adding on top of it is the fact that I can always be blocked from the website side.
With those conclusions I decided to download the entire HTML pages that the query returns with 2K chunks, using it I scraped 10K links of grow diaries into my first database.

In [20]:
# startpoint = 0
# count = 2000
# prefix_url = "https://growdiaries.com/explore"

# service = Service('./chromedriver.exe')
# op = webdriver.ChromeOptions()
# # op.add_argument('--headless')
# elements = []
# url_df = pd.DataFrame()

In [21]:
# driver = webdriver.Chrome(service=service, options=op)
# while startpoint < 10000:
#     query = f"?action=loadpage&category=all&tags=harvested&start={startpoint}&count={count}&undefined=&ajax=true&ust=9c824c5a63187dd5f579502e26ac3594c3dbb8752fd954432a4d6e1b0ff2e41f8654e9f3"
#     final_url = prefix_url + query
#     driver.get(final_url)
#     elements= driver.find_elements(By.XPATH, "//a[@class='name']")
#     startpoint += count
#     for element in elements:
#         temp_df = pd.DataFrame({"DiaryName": [element.text], "Url": [element.get_attribute('href')]})
#         url_df = pd.concat([url_df, temp_df], ignore_index=True)
    
# driver.quit()


In [22]:
# url_df.to_csv("Diary Links.csv", index=False)

### Step 2:
Now, when having the links for my desired database, I need to start collecting the raw data from each diary. As the first action toward the real data, my decision was to include the following as features:
diary_name<br> strain<br> strains_company<br> light_watt<br> nutrients<br> watering<br> soil<br> germination<br> grow_techniques<br>grow_room_size<br> weeks_to_harvest<br> num_of_plants<br> likes<br>comments<br> views<br> bud_dry_weight<br> bud_wet_weight

Using likes, comments, and views may help to increase/decrease the reliability of the data point.<br>
The method used to scrape the data is XPATH, it have conditional filtering and I found it as the perfect way to reach every selector I needed.

Highly recommand on: https://devhints.io/xpath for understanding and creating xpath queries

<b>Note: To make it esaier, I saved each time 1K of data in a file</b>

In [23]:
url_df = pd.read_csv("Diary Links.csv")

In [24]:
# driver = webdriver.Chrome(service=service, options=op)
# count = 2000
# const = 1000
# while count < 10000:
#     det_df = pd.DataFrame()
#     for j, url in enumerate(url_df["Url"][count:count+const]):
#         driver.get(url)
#         driver.maximize_window()
#         clickable = driver.find_elements(By.XPATH, "//div[@class='btn_less']")
#         click_btn(clickable)
#         report_items = driver.find_elements(By.XPATH, "//div[contains(@class,'report_items')]//*[@class='info']")
#         outcome = driver.find_elements(By.XPATH, "//div[contains(@class,'parameters_item')]")
#         likes = driver.find_element(By.XPATH, "//div[@class='report_statistic']//div[.//@class='icon-leaf-like']")
#         comments = driver.find_element(By.XPATH, "//div[@class='report_statistic']//div[.//@class='icon comment']")
#         views = driver.find_element(By.XPATH, "//div[@class='report_statistic']//div[.//@class='icon eye']")
#         d = {"diary_name": url_df['DiaryName'].loc[count+j], "strain": "", "strains_company":"", "light_watt": "", "nutrients": "", "watering": "", "soil": "", "germination": "", 'grow_techniques': "", "grow_room_size": "",
#         "weeks_to_harvest": "", "num_of_plants": "","likes": likes.text, "comments": comments.text,"views": views.text,"bud_dry_weight": "", "bud_wet_weight": ""}

        # for i, item in enumerate(report_items):
        #     try:
        #         text = item.text
        #         texts = text.split("\n")
        #         if i == 0:
        #             d["strain"] = texts[0]
        #             d['strains_company'] = texts[1]
        #         elif "LED" in text:
        #             d['light_watt'] += text.replace("\n", " ") + ","
        #         elif "Nutrients" in text:
        #             d['nutrients'] += texts[0] +","
        #         elif "Watering" in text:
        #             d['watering'] = texts[0]
        #         elif "Soil" in text or "Grow" in text:
        #             d['soil']+= text.replace("\n", " ")+","
        #         elif "Germination" in text:
        #             d['germination'] = texts[0]
        #         elif text.find("Week") > 0 and len(texts[0]) > 1:
        #             d['grow_techniques'] += texts[0] + ","
        #         elif text.find("Tent") > 0:
        #             d['grow_room_size'] = texts[0]
        #     except:
        #         pass
        # for i, details in enumerate(outcome):
        #     try:
        #         text = details.text
        #         texts = text.split("\n")
        #         if i == 0:
        #             d["weeks_to_harvest"] = texts[1]
        #         elif "BUD WET WEIGHT" in text:
        #             d['bud_wet_weight']= texts[1]
        #         elif "BUD DRY WEIGHT" in text:
        #             d['bud_dry_weight']= texts[1]
        #         elif "NUMBER OF PLANTS HARVESTED" in text:
        #             d['num_of_plants'] = texts[1].split(" ")[0]
        #         elif "TOTAL LIGHT POWER" in text:
        #             d['light_watt'] = texts[1]
        #         elif "GROW ROOM" in text:
        #             d['grow_room_size']= texts[1]
        #     except:
        #         pass
        # det_df = pd.concat([det_df,pd.DataFrame(d, index=[0])], ignore_index=True)
    # det_df.to_csv(f"Data_{count}_{count+const}.csv", index=False)
    # count += const

# driver.quit()

Connecting all files to one csv file:

In [25]:
# count = 0
# const = 1000
# full_df = pd.DataFrame()
# while count < 10000:
#     full_df = pd.concat([full_df, pd.read_csv(f"Data_{count}_{count+const}.csv")])
#     count += const
# full_df.to_csv("GrowDiariesRowData.csv", index=False)

### Initial analysis and data refinement: 
<small>Nulls: np.nan/NaN/None/NA</small><br>
Strating with testing which one of the dry/wet bud weight have less null values and set it as the target variable. <br>
After, I was looking for the number of null values in each feature nd wrote it into a dictionery, that's the way to find out the features contains null and how many.

In [26]:
full_df = pd.read_csv("GrowDiariesRowData.csv")
full_df['bud_dry_weight'].isna().sum(), full_df['bud_wet_weight'].isna().sum()

(5676, 4091)

In [27]:
full_df = full_df.drop("bud_dry_weight", axis=1)

In [28]:
# Showing all the features with nulls
def count_nulls(df):
    null_dict = {}
    for i in df.columns:
        if df[i].isnull().values.any():
            null_dict[i]=df[i].isnull().sum()
    return sorted(null_dict.items(), key=lambda x: x[1])
# Code partly from: https://stackoverflow.com/questions/29530232/how-to-check-if-any-value-is-nan-in-a-pandas-dataframe, https://www.codingem.com/python-sort-dictionary/,

count_nulls(full_df)

[('diary_name', 1),
 ('strain', 2),
 ('strains_company', 2),
 ('weeks_to_harvest', 3),
 ('watering', 489),
 ('soil', 858),
 ('light_watt', 1462),
 ('num_of_plants', 1621),
 ('nutrients', 2074),
 ('grow_techniques', 2397),
 ('grow_room_size', 3675),
 ('germination', 3995),
 ('bud_wet_weight', 4091)]

In [29]:
full_df = full_df.dropna(subset=['bud_wet_weight', 'grow_room_size', 'num_of_plants', 'light_watt'])
count_nulls(full_df), full_df.shape

([('watering', 117),
  ('soil', 252),
  ('nutrients', 560),
  ('grow_techniques', 864),
  ('germination', 1474)],
 (4218, 16))

The other features but gremination can be filled with the median, different category or any rational way that fits the each feature. For the germination I will look for the unique values and try to figure it out from it.

In [30]:
full_df['germination'].value_counts()

germination
Paper Towel              1001
Glass Of Water            647
Directly In Substrate     542
Other                     255
Peat Pellet               217
Rockwool Cube              82
Name: count, dtype: int64

In [31]:
full_df = full_df.drop("germination", axis=1)

As I see it, we have more null values than unique values so I'll drop it.

In [45]:
null_feat = dict(count_nulls(full_df)).keys()
null_feat

dict_keys(['soil', 'nutrients', 'grow_techniques'])

In [47]:
full_df['watering'].value_counts()

watering
Manual                                3703
Hydroponics                            325
Drip                                   157
Aeroponics                              32
Watering with Dirty Aquarium Water       1
Name: count, dtype: int64

In [40]:
# filling with the forward fill
full_df['watering'] = full_df['watering'].fillna(method='ffill')
full_df['watering'].value_counts()

watering
Manual                                3703
Hydroponics                            325
Drip                                   157
Aeroponics                              32
Watering with Dirty Aquarium Water       1
Name: count, dtype: int64

In [49]:
count_nulls(full_df)

[('soil', 252), ('nutrients', 560), ('grow_techniques', 864)]

The three remain features can be count, I think the mapping of these features will be done in this way